# 总结
1. 直观的基础的基线条件的应用使得递归计算刺水呈指数级别增长
2. 使用结果缓存进行处理，可以显著提高递归计算效率
3. 使用装饰器进行自动化的结果缓存也可以提高递归计算效率
4. 使用迭代的方式处理递归，是最高效率的方法，能用递归解决的问题，都可以用迭代解决
5. 使用生成器将递归的每一轮计算的值进行输出

## 1. 最直接的递归计算

fn() = f(n-1) + f(n-2)

In [1]:
def fib1(n: int) ->int:
    if n<2:
        return n
    else:
        return fib1(n-2) + fib1(n-1)


In [2]:
fib1(36)

14930352

我们发现fib1()计算35的时候，用时超过2.9s，如果尝试fib(50)，则程序会永远不终止，并可能栈溢出，一定是代码编写存在问题，我们尝试计算fib1()的计算次数

In [3]:
count = 0
def fib11(n: int) ->int:
    global count
    count += 1
    if n<2:
        return n
    else:
        return fib11(n-2) + fib11(n-1)

In [4]:
fib11(36), count

(14930352, 48315633)

共计算48315633次，时长为7.5s，比原来的2.9s多的原因有两个：增加了global变量的计算，global计算在堆区，fib11函数的计算在栈区，堆栈之间的交互也会消耗资源

## 2. 用结果缓存在减轻栈空间的消耗

In [5]:
from typing import Dict
memo: Dict[int, int] = {0:0, 1:1}

count = 0
def fib2(n: int)->int:
    global count
    count += 1
    if n not in memo:
        memo[n] = fib2(n-1) + fib2(n-2)
    return memo[n]

In [6]:
fib2(36), count

(14930352, 71)

计算接近0s，总共计算71次，使用结果缓存可以大大减少栈空间的消耗

## 3. 自动化的结果缓存

使用python自带的装饰器，可以自动为任何函数缓存结果

In [7]:
from functools import lru_cache

count = 0

@lru_cache(maxsize=None)
def fib3(n):
    global count
    count += 1
    if n<2:
        return n
    return fib3(n-2) + fib3(n-1)

In [8]:
fib3(36), count

(14930352, 37)

计算接近0s，总共计算37次，使用自动化的结果缓存可以大大减少栈空间的消耗，此处的count计算应该存在问题，具体看下文（待解决）

## 4. 使用迭代的方式解决递归问题

In [9]:
count = 0
def fib4(n):
    global count 
    if n == 0:
        count += 1
        return n
    last, next = 0, 1
    for _ in range(1, n):
        count += 1
        last, next = next, last + next
    return next

In [10]:
fib4(36), count

(14930352, 35)

计算接近0s，总共计算35次，使用迭代的方式解决递归问题，可以大大减少计算消耗

## 5. 比较

In [11]:
count = 0
fib2(100), count

(354224848179261915075, 129)

In [12]:
count = 0
fib3(100), count

(354224848179261915075, 64)

In [13]:
count = 0
fib4(100), count

(354224848179261915075, 99)

理论来讲99次应该是最低的迭代次数了，所以fib3()的计数可能跟调用装饰函数有关，导致了计数异常，待后续解答

## 6. 使用生成器生成fib数列

In [16]:
def fib6(n):
    yield 0
    if n>0: yield 1
    last, next = 0, 1
    for _ in range(1, n):
        last, next = next, last + next
        yield next

In [17]:
for id, value in enumerate(fib6(100)):
    if id > 98:
        print(value)

218922995834555169026
354224848179261915075
